In [13]:
target_cols = ['Glycohemoglobin',
       'Fasting glucose (mg/dL)', 'Fasting glucose (mmol/L)',
       'Insulin (pmol/L)', 'HDL cholesterol (mg/dL)',
       'HDL cholesterol (mmol/L)',
       'Total cholesterol (mmol/L)', 'Triglycerides (mmol/L)',
       'LDL cholesterol (mg/dL)', 'Trunk Fat (%)','Total Fat (%)']

# Step 2: Select the relevant columns for input features and target variables
input_cols = ['Weight (kg)', 
              'Height (cm)',
       'Leg Length (cm)', 
       'Arm Length (cm)', 
       'Arm Circumference (cm)',
       'Waist Circumference (cm)', 
       'Gender', 'Age (years)',
       'Vigorous work activity (Y/N)',
         'Minutes vigorous-intensity work',
       'Minutes moderate-intensity work',
       'Minutes walk/bicycle for transportation',
       'Minutes vigorous recreational activities',
       'Minutes sedentary activity', 
       'Hours watch TV or videos past 30 days',
       'Hours use computer past 30 days']

In [14]:
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import pickle
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error, explained_variance_score
import optuna
import pandas as pd

# Step 1: Load your dataset into a Pandas DataFrame
data = pd.read_csv('populated.csv')

# Step 3: Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(data[input_cols], data[target_cols], test_size=0.2, random_state=0)

# Step 4: Check and preprocess the target variable to handle invalid values
for col in target_cols:
    # Check for NaN or infinity values
    nan_indices = np.isnan(y_train[col])
    inf_indices = np.isinf(y_train[col])
    valid_indices = ~(nan_indices | inf_indices)

    # Remove invalid values
    X_train = X_train[valid_indices]
    y_train = y_train[valid_indices]
    
# Step 4: Scale the input features
scaler = StandardScaler()
scaler.fit(X_train[input_cols])
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

with open('scaler.pkl', 'wb') as file:
    pickle.dump(scaler, file)

In [15]:
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import pickle
from xgboost import XGBRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error, explained_variance_score
import optuna
import pandas as pd

# Create a dictionary to store the models and best parameters for each target column
models = {}
best_params = {}
best_rmse = {}
best_mae = {}

# Define the objective function for Optuna
def objective(trial):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 50, 200),
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.5),
        'max_depth': trial.suggest_int('max_depth', 3, 10)
    }

    model = XGBRegressor(random_state=0, **params)
    model.fit(X_train_scaled, y_train[col])

    y_pred = model.predict(X_test_scaled)
    rmse = np.sqrt(mean_squared_error(y_test[col], y_pred))

    return rmse

# Fit XGBoost models for each target column with hyperparameter tuning
for col in target_cols:
    study = optuna.create_study(direction='minimize')
    study.optimize(objective, n_trials=100, timeout=20)  # Adjust the timeout value according to your time-bound constraint

    best_params[col] = study.best_params
    best_model = XGBRegressor(random_state=0, **best_params[col])
    best_model.fit(X_train_scaled, y_train[col])
    models[col] = best_model

    # Print evaluation metrics
    y_pred = best_model.predict(X_test_scaled)
    rmse = np.sqrt(mean_squared_error(y_test[col], y_pred))
    mae = mean_absolute_error(y_test[col], y_pred)
    r2 = r2_score(y_test[col], y_pred)
    explained_var = explained_variance_score(y_test[col], y_pred)

    best_rmse[col] = rmse
    best_mae[col] = mae

    print(f"Column: {col}")
    print(f"RMSE: {rmse}")
    print(f"MAE: {mae}")
    print(f"R2: {r2}")
    print(f"Explained Variance: {explained_var}")
    print()

# Save the best model parameters for each column in a text file
with open('best_model_params.txt', 'w') as file:
    for col, params in best_params.items():
        file.write(f"Column: {col}\n")
        file.write(f"Best Parameters: {params}\n\n")

# Save the best RMSE and MAE for each column in a text file
with open('best_metrics.txt', 'w') as file:
    for col, rmse in best_rmse.items():
        file.write(f"Column: {col}\n")
        file.write(f"Best RMSE: {rmse}\n")
        file.write(f"Best MAE: {best_mae[col]}\n\n")

# Pickle the models and scaler
with open('models.pkl', 'wb') as file:
    pickle.dump(models, file)


[I 2023-06-30 08:27:20,394] A new study created in memory with name: no-name-2e6f3010-c832-46f8-8a32-9a7c7fcdeaf8
[I 2023-06-30 08:27:24,467] Trial 0 finished with value: 0.39512041129082653 and parameters: {'n_estimators': 168, 'learning_rate': 0.024415468685401863, 'max_depth': 10}. Best is trial 0 with value: 0.39512041129082653.
[I 2023-06-30 08:27:25,249] Trial 1 finished with value: 0.6714068444893633 and parameters: {'n_estimators': 51, 'learning_rate': 0.06015251528558077, 'max_depth': 7}. Best is trial 0 with value: 0.39512041129082653.
[I 2023-06-30 08:27:27,697] Trial 2 finished with value: 0.6881711408310122 and parameters: {'n_estimators': 149, 'learning_rate': 0.03704314732601082, 'max_depth': 5}. Best is trial 0 with value: 0.39512041129082653.
[I 2023-06-30 08:27:31,998] Trial 3 finished with value: 1.0000078103784409 and parameters: {'n_estimators': 172, 'learning_rate': 0.01167987192943215, 'max_depth': 7}. Best is trial 0 with value: 0.39512041129082653.
[I 2023-06-3

Column: Glycohemoglobin
RMSE: 0.3738854511667039
MAE: 0.2406635546418074
R2: 0.860721620512657
Explained Variance: 0.8608852129533108



[I 2023-06-30 08:27:55,033] Trial 0 finished with value: 6.862940052223307 and parameters: {'n_estimators': 195, 'learning_rate': 0.30621471958819, 'max_depth': 10}. Best is trial 0 with value: 6.862940052223307.
[I 2023-06-30 08:27:57,389] Trial 1 finished with value: 8.610272852719657 and parameters: {'n_estimators': 84, 'learning_rate': 0.3635401931868705, 'max_depth': 7}. Best is trial 0 with value: 6.862940052223307.
[I 2023-06-30 08:27:59,232] Trial 2 finished with value: 19.6872906747668 and parameters: {'n_estimators': 162, 'learning_rate': 0.2818492911994847, 'max_depth': 3}. Best is trial 0 with value: 6.862940052223307.
[I 2023-06-30 08:28:01,253] Trial 3 finished with value: 24.185933490396568 and parameters: {'n_estimators': 71, 'learning_rate': 0.03005702540412554, 'max_depth': 7}. Best is trial 0 with value: 6.862940052223307.
[I 2023-06-30 08:28:05,879] Trial 4 finished with value: 16.57228027798559 and parameters: {'n_estimators': 184, 'learning_rate': 0.03706927494031

Column: Fasting glucose (mg/dL)
RMSE: 6.862940052223307
MAE: 2.9855078449294323
R2: 0.9530391883587291
Explained Variance: 0.9530937908788663



[I 2023-06-30 08:28:23,400] Trial 0 finished with value: 1.3258639499709315 and parameters: {'n_estimators': 101, 'learning_rate': 0.019119686009535602, 'max_depth': 8}. Best is trial 0 with value: 1.3258639499709315.
[I 2023-06-30 08:28:26,377] Trial 1 finished with value: 0.6664707422563959 and parameters: {'n_estimators': 128, 'learning_rate': 0.16596299457923222, 'max_depth': 6}. Best is trial 1 with value: 0.6664707422563959.
[I 2023-06-30 08:28:28,329] Trial 2 finished with value: 0.9837642156025314 and parameters: {'n_estimators': 171, 'learning_rate': 0.4744181396352374, 'max_depth': 3}. Best is trial 1 with value: 0.6664707422563959.
[I 2023-06-30 08:28:29,851] Trial 3 finished with value: 1.0945935129251765 and parameters: {'n_estimators': 130, 'learning_rate': 0.40100327913792355, 'max_depth': 3}. Best is trial 1 with value: 0.6664707422563959.
[I 2023-06-30 08:28:30,997] Trial 4 finished with value: 1.6656928655893104 and parameters: {'n_estimators': 71, 'learning_rate': 0.

...

Column: LDL cholesterol (mg/dL)
RMSE: 16.92337719037189
MAE: 12.729094496186582
R2: 0.762112828787461
Explained Variance: 0.7621128288355276



[I 2023-06-30 08:31:39,933] Trial 0 finished with value: 0.9030975203845353 and parameters: {'n_estimators': 179, 'learning_rate': 0.2041600609992856, 'max_depth': 8}. Best is trial 0 with value: 0.9030975203845353.
[I 2023-06-30 08:31:45,716] Trial 1 finished with value: 1.0927755115347957 and parameters: {'n_estimators': 181, 'learning_rate': 0.1127310082624359, 'max_depth': 8}. Best is trial 0 with value: 0.9030975203845353.
[I 2023-06-30 08:31:47,694] Trial 2 finished with value: 2.692792338881918 and parameters: {'n_estimators': 178, 'learning_rate': 0.05976658404953586, 'max_depth': 3}. Best is trial 0 with value: 0.9030975203845353.
[I 2023-06-30 08:31:48,487] Trial 3 finished with value: 2.6927320830718586 and parameters: {'n_estimators': 53, 'learning_rate': 0.09640825370012306, 'max_depth': 4}. Best is trial 0 with value: 0.9030975203845353.
[I 2023-06-30 08:31:54,066] Trial 4 finished with value: 0.8100250051821997 and parameters: {'n_estimators': 152, 'learning_rate': 0.162

Column: Trunk Fat (%)
RMSE: 0.8100250051821997
MAE: 0.5157294107255387
R2: 0.9914947639079421
Explained Variance: 0.9914948219253704



[I 2023-06-30 08:32:02,406] Trial 0 finished with value: 15.852409044439534 and parameters: {'n_estimators': 66, 'learning_rate': 0.011412276910268169, 'max_depth': 8}. Best is trial 0 with value: 15.852409044439534.
[I 2023-06-30 08:32:04,892] Trial 1 finished with value: 1.851519659009503 and parameters: {'n_estimators': 124, 'learning_rate': 0.25128765354194765, 'max_depth': 5}. Best is trial 1 with value: 1.851519659009503.
[I 2023-06-30 08:32:08,162] Trial 2 finished with value: 5.33913551886877 and parameters: {'n_estimators': 94, 'learning_rate': 0.020402540834086543, 'max_depth': 9}. Best is trial 1 with value: 1.851519659009503.
[I 2023-06-30 08:32:09,506] Trial 3 finished with value: 2.18719261348133 and parameters: {'n_estimators': 90, 'learning_rate': 0.37538986530216445, 'max_depth': 4}. Best is trial 1 with value: 1.851519659009503.
[I 2023-06-30 08:32:10,834] Trial 4 finished with value: 2.814448544474159 and parameters: {'n_estimators': 57, 'learning_rate': 0.0536190168

Column: Total Fat (%)
RMSE: 0.8097163372146976
MAE: 0.528037032159889
R2: 0.9903154292999047
Explained Variance: 0.990316953358431

